# Implementación de la autoatención con pesos entrenables

En  esta  sección,  implementamos  el  mecanismo  de  autoatención  utilizado  en  la  arquitectura  original  del  transformador,  los  modelos  GPT  y  la  mayoría  de  los  demás  LLM  populares.  

Este  mecanismo  también  se  denomina  atención  escalar  de  producto  escalar.

![Texto alternativo](./imgs/3.12.png)

- La self-attention con pesos entrenables calcula vectores de contexto como combinaciones ponderadas de las entradas.

- La diferencia clave respecto a la versión básica: se introducen matrices de pesos entrenables que se ajustan durante el entrenamiento.

- Estas matrices permiten que el modelo aprenda a producir buenos vectores de contexto.

## Cálculo de los pesos de atención paso a paso

En este apartado se implementa el mecanismo de autoatención introduciendo tres matrices de peso entrenables: `Wq`, `Wk` y `Wv`. Estas matrices se utilizan para proyectar los *tokens* de entrada incrustados `x(i)` en vectores de **consulta (query)**, **clave (key)** y **valor (value)**, respectivamente. 

El procedimiento comienza calculando los vectores `q`, `k` y `v` mediante multiplicaciones de la entrada por las correspondientes matrices de peso. De forma análoga a lo visto en la sección anterior, primero se ilustra el cálculo de un único vector de contexto `z(i)`. Posteriormente, el método se generaliza para obtener todos los vectores de contexto de la secuencia.  

![Texto alternativo](./imgs/3.13.png)

In [1]:
import torch

#Considerar  la  siguiente  oración  de  entrada,  que  ya  ha  sido  incorporada  en  vectores  tridimensionales 
inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your     
    [0.55, 0.87, 0.66], # journey  (x^2)
    [0.57, 0.85, 0.64], # starts   (x^3)
    [0.22, 0.58, 0.33], # with     
    [0.77, 0.25, 0.10], # one      
    [0.05, 0.80, 0.55]] # step     
)

In [7]:
x_2 = inputs[1] #segundo elemento de la entrada
d_in = inputs.shape[1] #tamaño de incrustación de entrada
d_out = 2 #tamaño de incrustación de salida

#Inicializar tres  matrices  de  peso  Wq ,  Wk  y  Wv

torch.manual_seed(123)
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False) #reducir el desorden en las salidas
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

#Calcular los vectores de consulta, clave y valor 
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value

print(query_2) #vector bidimensional


tensor([0.4306, 1.4551])


Los  parámetros  de  peso  son  los  coeficientes  fundamentales  aprendidos  que  definen  las  
conexiones  de  la  red,  mientras  que  los  pesos  de  atención  son  dinámicos  y  específicos  del contexto.

Aunque el objetivo es z_2, z requiere de los vectores de clave y valor, para todos los elemnentos de entradas, ya que están involucrados en el cálculo.

In [ ]:
keys = inputs @ W_key
values = inputs @ W_value